<a href="https://colab.research.google.com/github/ekacs/simadu_analytical_insight/blob/main/analisa_kinerja_TL_pengaduan_(01012019_07062022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persiapan
bahan analisa dari dbsimadu

Export dbsimadu to csv, saving on My Drive/Pokja SIMADU 2022/Simadu jango 2022

syntax querinya:

SELECT t_pengaduan.kode, t_pengaduan.ref_username, t_status_history.ref_username, t_status_history.pic, t_pengaduan.ref_admin, t_pengaduan.insert_datetime, t_pengaduan.update_user, t_status_history.update_datetime, tr_status.detil, tr_jenis_aduan.detil, tr_unit.detil FROM t_pengaduan LEFT JOIN t_status_history ON t_pengaduan.id = t_status_history.ref_id LEFT JOIN tr_status ON t_pengaduan.ref_status = tr_status.id LEFT JOIN tr_jenis_aduan ON t_pengaduan.ref_jenis_aduan = tr_jenis_aduan.id LEFT JOIN tr_unit ON t_pengaduan.ref_unit_kejadian = tr_unit.id WHERE insert_datetime>2018 AND ref_admin<>'NULL' ORDER BY insert_datetime ASC;

https://drive.google.com/file/d/1JzWv4O13ZbJTlELwo9QwJN5SuQoan0ia/view?usp=sharing
filename : t_pengaduan.csv

SELECT t_pengaduan.kode, t_pengaduan.ref_username, t_status_history.ref_username, t_status_history.pic, t_pengaduan.ref_admin, t_pengaduan.insert_datetime, t_pengaduan.update_user, t_status_history.update_datetime, tr_status.detil, tr_jenis_aduan.detil, tr_unit.detil FROM t_pengaduan LEFT JOIN t_status_history ON t_pengaduan.id = t_status_history.ref_id LEFT JOIN tr_status ON t_pengaduan.ref_status = tr_status.id LEFT JOIN tr_jenis_aduan ON t_pengaduan.ref_jenis_aduan_admin = tr_jenis_aduan.id LEFT JOIN tr_unit ON t_pengaduan.ref_unit_terlapor_admin = tr_unit.id WHERE insert_datetime>2018 AND ref_admin<>'NULL' ORDER BY insert_datetime ASC;

https://drive.google.com/open?id=1K-SCvEPBjvvgRvBH3PN37_MR3uqU66M3&authuser=auditorbodoh%40gmail.com&usp=drive_fs
filename: t_pengaduan_rev.csv


# koding awal

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import gzip
#pre-start
from google.colab import drive
drive.mount('/content/drive')
datas = pd.read_csv('/content/drive/My Drive/Pokja SIMADU 2022/Simadu jango 2022/t_pengaduan_rev.csv') #nama_file sesuaikan dengan nama file yang diupload
#datas.describe()
#datas.info()

datas.columns = ['kd_track', 'kd_pengadu', 'verifikator', 'auto info', 'korwil', 'update_datetime', 'auditor_korwil', 'insert_datetime', 'status_pengaduan', 'jenis_pengaduan', 'unit_eselon1']
#datas.head(10)
from datetime import timedelta
datas['update_datetime'] = pd.to_datetime(datas['update_datetime'])
datas['insert_datetime'] = pd.to_datetime(datas['insert_datetime'])
datas['lama_TL']=(datas['update_datetime']-datas['insert_datetime'])
#datas.info()
datas2 = datas[(datas.verifikator != 'admin')]
datas2.sort_values(by=['lama_TL'], ascending=True)
datas3 = datas2.drop_duplicates(subset=['kd_track','kd_pengadu'], keep='last')
datas3.info()

#analisa1
#status pengaduan per korwil dr th 2019 sd terakhir

#sstatus pengaduan per korwil yang masuk tahun 2019
korwil_2019 = datas3[(datas3['insert_datetime'] >= "2019-01-01") & (datas3['insert_datetime']< "2020-01-01")]
korwil_2019a = korwil_2019.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#sstatus pengaduan per korwil yang masuk tahun 2020
korwil_2020 = datas3[(datas3['insert_datetime'] >= "2020-01-01") & (datas3['insert_datetime']< "2021-01-01")]
korwil_2020a = korwil_2020.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#sstatus pengaduan per korwil yang masuk tahun 2021
korwil_2021 = datas3[(datas3['insert_datetime'] >= "2021-01-01") & (datas3['insert_datetime']< "2022-01-01")]
korwil_2021a = korwil_2021.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#sstatus pengaduan per korwil yang masuk tahun 2022
korwil_2022 = datas3[(datas3['insert_datetime'] >= "2022-01-01") & (datas3['insert_datetime']< "2023-01-01")]
korwil_2022a = korwil_2022.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#analisa2
#performa korwil thd pengaduan masuk di 2019
korwil_2019b = korwil_2019[(korwil_2019.status_pengaduan != 'Baru') & (korwil_2019.status_pengaduan != 'Proses')]
korwil_2019b = korwil_2019b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})
#Rata2 performa korwil thd pengaduan masuk di 2020
korwil_2020b = korwil_2020[(korwil_2020.status_pengaduan != 'Baru') & (korwil_2020.status_pengaduan != 'Proses')]
korwil_2020b = korwil_2020b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})
#Rata2 performa korwil thd pengaduan masuk di 2021
korwil_2021b = korwil_2021[(korwil_2021.status_pengaduan != 'Baru') & (korwil_2021.status_pengaduan != 'Proses')]
korwil_2021b = korwil_2021b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})
#Rata2 performa korwil thd pengaduan masuk di 2022
korwil_2022b = korwil_2022[(korwil_2022.status_pengaduan != 'Baru') & (korwil_2022.status_pengaduan != 'Proses')]
korwil_2022b = korwil_2022b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})


Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 2 to 1715
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   kd_track          396 non-null    object         
 1   kd_pengadu        396 non-null    object         
 2   verifikator       396 non-null    object         
 3   auto info         357 non-null    object         
 4   korwil            396 non-null    object         
 5   update_datetime   396 non-null    datetime64[ns] 
 6   auditor_korwil    396 non-null    object         
 7   insert_datetime   396 non-null    datetime64[ns] 
 8   status_pengaduan  396 non-null    object         
 9   jenis_pengaduan   396 non-null    object         
 10  unit_eselon1      396 non-null    object         
 11  lama_TL           396 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), object(9), timedelta64[ns](1)
memory usage: 40.2+ KB


#data tabel terperincinya
(awal 2019 sd terakhir)
sumber analisa

In [2]:
datas3

,kd_track,kd_pengadu,verifikator,auto info,korwil,update_datetime,auditor_korwil,insert_datetime,status_pengaduan,jenis_pengaduan,unit_eselon1,lama_TL
2,xSndY,fendiya,198212032006041001,File Bukti telah diupload untuk melengkapi Pel...,korwil2,2019-06-06 06:30:43,198212032006041001,2020-07-13 08:00:24,Selesai,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-404 days +22:30:19
4,IFsJL,galih,198212032006041001,198701082009122006 telah menjadi verifikator u...,korwil3,2019-06-30 22:16:15,198212032006041001,2020-08-19 18:57:59,Baru,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-416 days +03:18:16
6,TuL3e,ferry67,198212032006041001,NaN,korwil2,2019-07-09 00:48:30,198212032006041001,2019-08-12 10:05:37,Proses,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-35 days +14:42:53
8,4O4UI,mahrusali85,198212032006041001,NaN,korwil2,2019-07-09 13:40:56,198212032006041001,2019-08-12 10:05:04,Proses,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-34 days +03:35:52
10,84fKT,haryo77,198705252018012001,NaN,korwil4,2019-07-16 15:12:56,198705252018012001,2019-08-07 16:06:39,Proses,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-23 days +23:06:17
...,...,...,...,...,...,...,...,...,...,...,...,...
1705,rdlny,izharsyah,korwil4,Dilimpahkan ke Inspektur Daerah Kabupaten Kuta...,korwil4,2022-05-31 21:13:46,korwil4,2022-06-06 12:54:39,Dilimpahkan,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-6 days +08:19:07
1708,aPWb1,zhephyra2005,korwil3,NaN,korwil3,2022-06-03 11:30:46,korwil3,2022-06-06 11:06:09,Proses,Pengaduan Kepegawaian,Direktorat Jenderal Perhubungan Udara,-3 days +00:24:37
1710,VIOpx,operator2,19850212,korwil4 telah menjadi verifikator untuk aduan ...,korwil4,2022-06-06 11:20:49,korwil4,2022-06-06 16:21:23,Ditolak,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-1 days +18:59:26
1713,QxMMd,danang,korwil4,Dilimpahkan ke Otoritas Bandara Wilayah III Su...,korwil4,2022-06-06 11:39:52,korwil4,2022-06-10 09:20:08,Dilimpahkan,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,-4 days +02:19:44


# analisa 1
statistik data status pengaduan yang masuk

In [3]:
korwil_2019a

status_pengaduan
korwil  status_pengaduan                  
korwil2 Ditolak                          6
        Proses                           3
        Selesai                          2
korwil3 Proses                           2
        Selesai                          1
korwil4 Proses                           5

In [4]:
korwil_2020a

status_pengaduan
korwil  status_pengaduan                  
korwil1 Dilimpahkan                      2
        Ditolak                          7
        Proses                           4
        Selesai                         10
korwil2 Ditolak                          1
        Proses                           2
        Selesai                         28
korwil3 Baru                            11
        Proses                           1
        Selesai                         41
korwil4 Ditolak                          2
        Selesai                         17
korwil5 Ditolak                          2
        Proses                           2
        Selesai                         11

In [5]:
korwil_2021a

status_pengaduan
korwil  status_pengaduan                  
korwil1 Ditolak                          1
        Selesai                         16
korwil2 Dilimpahkan                      1
        Ditolak                          4
        Proses                           1
        Selesai                         23
korwil3 Dilimpahkan                      4
        Ditolak                          2
        Proses                           7
        Selesai                         21
korwil4 Dilimpahkan                      1
        Ditolak                          7
        Proses                           1
        Selesai                         35
korwil5 Dilimpahkan                      3
        Ditolak                          1
        Proses                           3
        Selesai                         23

In [6]:
korwil_2022a

status_pengaduan
korwil  status_pengaduan                  
korwil1 Dilimpahkan                      2
        Proses                           6
        Selesai                          6
korwil2 Baru                             1
        Dilimpahkan                      1
        Ditolak                          1
        Proses                           7
        Selesai                         13
korwil3 Dilimpahkan                      3
        Ditolak                          1
        Proses                           7
        Selesai                          3
korwil4 Dilimpahkan                      5
        Ditolak                          5
        Proses                           3
        Selesai                          5
korwil5 Dilimpahkan                      2
        Ditolak                          2
        Proses                           4
        Selesai                          5

# analisa 2
rata2 performa tiap korwil menindaklanjuti pengaduan per status 

In [7]:
korwil_2019b

lama_TL
korwil  status_pengaduan                             
korwil2 Ditolak          -14 days +15:47:40.833333334
        Selesai                    -57 days +05:10:10
korwil3 Selesai                    -46 days +05:51:14

In [8]:
korwil_2020b

lama_TL
korwil  status_pengaduan                              
korwil1 Dilimpahkan         -316 days +00:56:30.500000
        Ditolak           -70 days +21:20:48.142857143
        Selesai             -118 days +03:00:04.200000
korwil2 Ditolak                     -65 days +06:09:52
        Selesai          -131 days +16:45:34.964285714
korwil3 Selesai          -117 days +15:39:01.243902440
korwil4 Ditolak                     -19 days +05:39:23
        Selesai          -146 days +05:03:33.764705882
korwil5 Ditolak             -181 days +15:09:49.500000
        Selesai          -139 days +14:03:27.181818182

In [9]:
korwil_2021b

lama_TL
korwil  status_pengaduan                              
korwil1 Ditolak                      -1 days +22:37:22
        Selesai              -90 days +01:05:40.500000
korwil2 Dilimpahkan                 -29 days +04:33:51
        Ditolak               -9 days +14:36:34.500000
        Selesai          -145 days +14:07:28.521739130
korwil3 Dilimpahkan                 -10 days +07:19:54
        Ditolak               -1 days +04:32:01.500000
        Selesai          -110 days +01:47:31.238095238
korwil4 Dilimpahkan                  -1 days +12:39:09
        Ditolak            -5 days +11:59:17.714285715
        Selesai          -118 days +01:15:47.342857142
korwil5 Dilimpahkan       -13 days +04:10:08.333333334
        Ditolak                     -24 days +07:48:28
        Selesai          -236 days +12:59:18.043478260

In [10]:
korwil_2022b

lama_TL
korwil  status_pengaduan                              
korwil1 Dilimpahkan                 -14 days +11:31:13
        Selesai           -87 days +14:48:56.333333333
korwil2 Dilimpahkan                  -1 days +23:56:47
        Ditolak                      -9 days +02:17:30
        Selesai          -177 days +15:01:43.538461538
korwil3 Dilimpahkan        -9 days +10:01:35.666666667
        Ditolak                      -5 days +13:35:37
        Selesai                    -140 days +13:43:52
korwil4 Dilimpahkan           -5 days +06:11:10.600000
        Ditolak                      -1 days +03:35:41
        Selesai                     -50 days +17:21:56
korwil5 Dilimpahkan          -16 days +03:20:54.500000
        Ditolak              -27 days +09:23:40.500000
        Selesai             -100 days +08:31:02.600000

# export to CSV


In [11]:
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/My Drive/Pokja SIMADU 2022/Simadu jango 2022/'
datas3.to_csv(path+'analisa kinerja TL pengaduan (01012019-07062022).csv', index=None)


Mounted at /content/drive


# export to google-sheet
bakalan jadi bahan olah google-studio


In [12]:
#key = 1xJGScFPlQb7faPApmzWBnEV7PSIvGiVC_rkfGpAjJ94
#baca deh https://docs.gspread.org/en/latest/index.html


In [13]:
#!pip install --upgrade gspread

In [14]:
#!pip install gspread-dataframe

In [15]:
#!pip install --upgrade --quiet gspread

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Go to https://sheets.google.com to see your new spreadsheet.


from gspread_dataframe import get_as_dataframe, set_with_dataframe

sh = gc.create('analisa_kinerja_TL_pengaduan_rev')
#sh = gc.create(brandname + 'analisa_kinerja_TL_pengaduan_rev')
worksheet = gc.open('analisa_kinerja_TL_pengaduan_rev').sheet1

set_with_dataframe(worksheet, datas3)
